In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os

# Load mô hình đã huấn luyện (.h5)
model = tf.keras.models.load_model("model_mobilenetv2.h5")

# Thư mục chứa các thư mục con đại diện cho từng người
test_dir = "C:/Users/Loc/Desktop/DACS_git/Do_An_Co_So/code/code_final/data_split/test"
class_names = sorted(os.listdir(test_dir))  # List các tên người
IMG_SIZE = (160, 160)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model_mobilenetv2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Tải Haar Cascade để phát hiện khuôn mặt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Mở webcam
cap = cv2.VideoCapture(0)
print("[INFO] Webcam đang mở. Nhấn 'q' để thoát.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        try:
            face_resized = cv2.resize(face, IMG_SIZE)
        except:
            continue

        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=0)

        # Dự đoán nhãn
        predictions = model.predict(face_input)
        pred_index = np.argmax(predictions[0])
        confidence = predictions[0][pred_index]

        label = class_names[pred_index]
        label_text = f"{label} ({confidence*100:.2f}%)"

        # Vẽ khung và label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label_text, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    cv2.imshow("Face Recognition (MobileNetV2 .h5)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
